# Training on Self-defined Model

* 本代码对应笔记（四）：自定义模型训练

In [2]:
import tensorflow as tf
import numpy as np

## 自动求导机制

### Experiment 1：求导机制的简单例子

In [4]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x) # watch:确保某个Tensor被tape追踪
    y = x * x
dy_dx = g.gradient(y, x) # gradient:根据tape计算某个或某些Tensor的梯度
dy_dx

<tf.Tensor: id=11, shape=(), dtype=float32, numpy=6.0>

## 自定义模型训练

### Experiment 2：使用 GradientTape 自定义训练模型

#### 构建模型和参数

In [5]:
class MyModel(tf.keras.Model):

    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # 定义自己需要的层
        self.dense_1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense_2 = tf.keras.layers.Dense(num_classes)

    def call(self, inputs):
        # 定义前向传播
        # 使用在 (in `__init__`)定义的层
        x = self.dense_1(inputs)
        return self.dense_2(x)

In [6]:
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

In [7]:
model = MyModel(num_classes=10)

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = tf.keras.losses.CategoricalCrossentropy()

# Prepare the training dataset.
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((data, labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

#### 自定义训练

In [8]:
epochs = 3
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))
    
    # 遍历数据集的batch_size
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        
        # 打开GradientTape以记录正向传递期间运行的操作，这将启用自动区分。
        with tf.GradientTape() as tape:

            # 运行该模型的前向传播。 模型应用于其输入的操作将记录在GradientTape上。
            logits = model(x_batch_train, training=True)  # 这个minibatch的预测值

            # 计算这个minibatch的损失值
            loss_value = loss_fn(y_batch_train, logits)

        # 使用GradientTape自动获取可训练变量相对于损失的梯度。
        grads = tape.gradient(loss_value, model.trainable_weights)

        # 通过更新变量的值来最大程度地减少损失，从而执行梯度下降的一步。
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # 每200 batches打印一次.
        if step % 200 == 0:
            print('Training loss (for one batch) at step %s: %s' % (step, float(loss_value)))
            print('Seen so far: %s samples' % ((step + 1) * 64))

Start of epoch 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Training loss (for one batch) at step 0: 24.458969116210938
Seen so far: 64 samples
Start of epoch 1
Training loss (for one batch) at step 0: 19.38613510131836
Seen so far: 64 samples
Start of epoch 2
Training loss (for one batch) at step 0: 17.059972763061523
Seen so far: 64 samples


### Experiment 3：自定义模型进阶（加入评估函数）

#### 构建模型

In [9]:
class MyModel(tf.keras.Model):

    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        # 定义自己需要的层
        self.dense_1 = tf.keras.layers.Dense(32, activation='relu')
        self.dense_2 = tf.keras.layers.Dense(num_classes)
    
    def call(self, inputs):
        #定义前向传播
        # 使用在 (in `__init__`)定义的层
        x = self.dense_1(inputs)
        return self.dense_2(x)

#### 编译模型

In [10]:
x_train = np.random.random((1000, 32))
y_train = np.random.random((1000, 10))
x_val = np.random.random((200, 32))
y_val = np.random.random((200, 10))
x_test = np.random.random((200, 32))
y_test = np.random.random((200, 10))

#### 定义参数

In [11]:
# 优化器
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
# 损失函数
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

# 准备metrics函数
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
val_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# 准备训练数据集
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# 准备测试数据集
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(64)

#### 循环训练

In [12]:
model = MyModel(num_classes=10)
epochs = 3
for epoch in range(epochs):
    print('Start of epoch %d' % (epoch,))

    # 遍历数据集的batch_size
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
                
        # 对每一个batch
        with tf.GradientTape() as tape:
            logits = model(x_batch_train)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights)) # 通过优化器更新权重

        # 更新训练集的metrics
        train_acc_metric(y_batch_train, logits) # 更新训练集的logits     
            
    # 在每个epoch结束时显示metrics。
    train_acc = train_acc_metric.result()
    print('Training acc over epoch: %s' % (float(train_acc),))
    
    # 在每个epoch结束时重置训练指标
    train_acc_metric.reset_states() # 非常重要，不能漏掉！

    # 在每个epoch结束时运行一个验证集。
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val)
        # 更新验证集metrics
        val_acc_metric(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    print('Validation acc: %s' % (float(val_acc),))
    val_acc_metric.reset_states()

Start of epoch 0

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Training acc over epoch: 0.10100000351667404
Validation acc: 0.0949999988079071
Start of epoch 1
Training acc over epoch: 0.09799999743700027
Validation acc: 0.0949999988079071
Start of epoch 2
Training acc over epoch: 0.09600000083446503
Validation acc: 0.09000000357627869
